# Using Feature Store

## Learning Objective

In this notebook, you will learn how to:

* Import your features into Feature Store.
* Serve online prediction requests using the imported features.
* Access imported features in offline jobs, such as training jobs.

## Introduction

In this notebook, you will learn how to use Feature Store, a managed cloud service for machine learning engineers and data scientists to store, serve, manage and share machine learning features at a large scale.

Each learning objective will correspond to a __#TODO__ in the [student lab notebook](../labs/6_gapic_feature_store.ipynb) -- try to complete that notebook first before reviewing this solution notebook. 

This notebook assumes that you understand basic Google Cloud concepts such as [Project](https://cloud.google.com/storage/docs/projects), [Storage](https://cloud.google.com/storage) and [Vertex AI](https://cloud.google.com/vertex-ai/docs). Some machine learning knowledge is also helpful but not required.

### Dataset

This notebook uses a movie recommendation dataset as an example throughout all the sessions. The task is to train a model to predict if a user is going to watch a movie and serve this model online. 

**Make sure to enable the Vertex AI, Cloud Storage, and Compute Engine APIs.**

### Install additional packages

For this notebook, you need the Vertex SDK for Python.

In [1]:
# Setup your dependencies
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
# Upgrade the specified package to the newest available version
! pip3 install {USER_FLAG} --upgrade git+https://github.com/googleapis/python-aiplatform.git@main-test

  Cloning https://github.com/googleapis/python-aiplatform.git (to revision main-test) to /tmp/pip-req-build-wm84ezm8
  Running command git clone --filter=blob:none -q https://github.com/googleapis/python-aiplatform.git /tmp/pip-req-build-wm84ezm8
  Running command git checkout -b main-test --track origin/main-test
  Switched to a new branch 'main-test'
  Branch 'main-test' set up to track remote branch 'main-test' from 'origin'.
  Resolved https://github.com/googleapis/python-aiplatform.git to commit 59808e63aa07a725de03721ecdd4efef9e1b33e2
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 93 kB 1.7 MB/s             
     |████████████████████████████████| 152 kB 22.9 MB/s            
  Created wheel for google-cloud-aiplatform: filename=google_cloud_aiplatform-0.7.1-py2.py3-none-any.whl size=1696810 sha256=c7f7592b798b8956fd0909a0f6685e0a317b980909919c4c108fc9dbfad521ca
  Stored in directory: /tmp/pip-ephem-wheel-cache-klf3ea77/wheels/17/ff/e0/00d0fec24b

**Note**: Please ignore any incompatibility warnings and errors.


### Restart the kernel

After you install the SDK, you need to restart the notebook kernel so it can find the packages. You can restart kernel from *Kernel -> Restart Kernel*, or running the following:

In [1]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

Update `YOUR-PROJECT-ID` with your Project ID. **If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [2]:
import os

PROJECT_ID = "YOUR-PROJECT-ID"

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    # PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  qwiklabs-gcp-01-3ea79ecd1cfe


Otherwise, set your project ID here.

In [3]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "YOUR-PROJECT-ID"  # @param {type:"string"}

In [4]:
# Authenticate your google Cloud account
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

## Prepare for output

### Step 1. Create dataset for output

You need a BigQuery dataset to host the output data in `us-central1`. Input the name of the dataset you want to create and specify the name of the table you want to store the output later. These will be used later in the notebook.

**Make sure that the table name does NOT already exist**.


In [5]:
from datetime import datetime

from google.cloud import bigquery

In [6]:
# Output dataset
DESTINATION_DATA_SET = "movie_predictions"  # @param {type:"string"}
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DESTINATION_DATA_SET = "{prefix}_{timestamp}".format(
    prefix=DESTINATION_DATA_SET, timestamp=TIMESTAMP
)

# Output table. Make sure that the table does NOT already exist; the BatchReadFeatureValues API cannot overwrite an existing table
DESTINATION_TABLE_NAME = "training_data"  # @param {type:"string"}

DESTINATION_PATTERN = "bq://{project}.{dataset}.{table}"
DESTINATION_TABLE_URI = DESTINATION_PATTERN.format(
    project=PROJECT_ID, dataset=DESTINATION_DATA_SET, table=DESTINATION_TABLE_NAME
)

In [7]:
# Create dataset
REGION = "us-central1"  # @param {type:"string"}
client = bigquery.Client()
dataset_id = "{}.{}".format(client.project, DESTINATION_DATA_SET)
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset = client.create_dataset(dataset, timeout=30)
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset qwiklabs-gcp-01-3ea79ecd1cfe.movie_predictions_20210824110930


### Import libraries and define constants

In [11]:
# Copy all required files in your bucket.
# Make sure to replace your bucket name here.
!gsutil cp -r gs://cloud-training/mlongcp/v3.0_MLonGC/toy_data/feature_stores/* gs://<Your-bucket-name>

Copying gs://cloud-training/mlongcp/v3.0_MLonGC/toy_data/feature_stores/movie_prediction_toy.csv [Content-Type=text/csv]...
Copying gs://cloud-training/mlongcp/v3.0_MLonGC/toy_data/feature_stores/movie_prediction_toy.csv [Content-Type=text/csv]...
Copying gs://cloud-training/mlongcp/v3.0_MLonGC/toy_data/feature_stores/movie_prediction_toy.csv [Content-Type=text/csv]...
/ [3 files][  1.5 KiB/  1.5 KiB]                                                
Operation completed over 3 objects/1.5 KiB.                                      


In [8]:
# Other than project ID and featurestore ID and endpoints needs to be set.
# Make sure to replace your bucket name here.
API_ENDPOINT = "us-central1-aiplatform.googleapis.com"  # @param {type:"string"}
INPUT_CSV_FILE = "gs://<Your-bucket-name>/movie_prediction_toy.csv"

In [9]:
from google.cloud.aiplatform_v1beta1 import (
    FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient)
from google.cloud.aiplatform_v1beta1.types import FeatureSelector, IdMatcher
from google.cloud.aiplatform_v1beta1.types import \
    entity_type as entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import feature as feature_pb2
from google.cloud.aiplatform_v1beta1.types import \
    featurestore as featurestore_pb2
from google.cloud.aiplatform_v1beta1.types import \
    featurestore_monitoring as featurestore_monitoring_pb2
from google.cloud.aiplatform_v1beta1.types import \
    featurestore_online_service as featurestore_online_service_pb2
from google.cloud.aiplatform_v1beta1.types import \
    featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.protobuf.duration_pb2 import Duration

# Create admin_client for CRUD and data_client for reading feature values.
admin_client = FeaturestoreServiceClient(client_options={"api_endpoint": API_ENDPOINT})
data_client = FeaturestoreOnlineServingServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

# Represents featurestore resource path.
BASE_RESOURCE_PATH = admin_client.common_location_path(PROJECT_ID, REGION)

## Terminology and Concept

### Featurestore Data model

Feature Store organizes data with the following 3 important hierarchical concepts:
```
Featurestore -> EntityType -> Feature
```
* **Featurestore**: the place to store your features
* **EntityType**: under a Featurestore, an *EntityType* describes an object to be modeled, real one or virtual one.
* **Feature**: under an EntityType, a *feature* describes an attribute of the EntityType

In the movie prediction example, you will create a featurestore called *movie_prediction*. This store has 2 entity types: *Users* and *Movies*. The Users entity type has the age, gender, and like genres features. The Movies entity type has the genres and average rating features.


## Create Featurestore and Define Schemas

### Create Featurestore

The method to create a featurestore returns a
[long-running operation](https://google.aip.dev/151) (LRO). An LRO starts an asynchronous job. LROs are returned for other API
methods too, such as updating or deleting a featurestore. Calling
`create_fs_lro.result()` waits for the LRO to complete.

In [10]:
FEATURESTORE_ID = "movie_prediction_{timestamp}".format(timestamp=TIMESTAMP)
create_lro = admin_client.create_featurestore(
    featurestore_service_pb2.CreateFeaturestoreRequest(
        parent=BASE_RESOURCE_PATH,
        featurestore_id=FEATURESTORE_ID,
        featurestore=featurestore_pb2.Featurestore(
            display_name="Featurestore for movie prediction",
            online_serving_config=featurestore_pb2.Featurestore.OnlineServingConfig(
                fixed_node_count=3
            ),
        ),
    )
)

In [11]:
# Wait for LRO to finish and get the LRO result.
print(create_lro.result())

name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930"



You can use [GetFeaturestore](https://cloud.google.com/vertex-ai/docs/featurestore) or [Featurestores](https://cloud.google.com/vertex-ai/docs/featurestore/managing-featurestores) to check if the Featurestore was successfully created. The following example gets the details of the Featurestore.


In [12]:
admin_client.get_featurestore(
    name=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID)
)

name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930"
create_time {
  seconds: 1629803788
  nanos: 111037000
}
update_time {
  seconds: 1629803788
  nanos: 187222000
}
etag: "AMEw9yPmZHh0tF82VsW42YlQCrGd-AbPbMYwR6_WS0tHdthJCHIIixMkivjGGFGko14="
online_serving_config {
  fixed_node_count: 3
}
state: STABLE

### Create Entity Type
You can specify a monitoring config which will by default be inherited by all Features under this EntityType.

In [13]:
# Create users entity type with monitoring enabled.
# All Features belonging to this EntityType will by default inherit the monitoring config.
users_entity_type_lro = admin_client.create_entity_type(
    featurestore_service_pb2.CreateEntityTypeRequest(
        parent=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        entity_type_id="users",
        entity_type=entity_type_pb2.EntityType(
            description="Users entity",
            monitoring_config=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig(
                snapshot_analysis=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                    monitoring_interval=Duration(seconds=86400),  # 1 day
                ),
            ),
        ),
    )
)

# Similarly, wait for EntityType creation operation.
print(users_entity_type_lro.result())

name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/users"
etag: "AMEw9yNw3iflwyrbZyyI03Vx09aaI-KAS214bwrKzEGvEF3RSClb"



In [14]:
# Create movies entity type without a monitoring configuration.
movies_entity_type_lro = admin_client.create_entity_type(
    featurestore_service_pb2.CreateEntityTypeRequest(
        parent=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        entity_type_id="movies",
        entity_type=entity_type_pb2.EntityType(description="Movies entity"),
    )
)

# Similarly, wait for EntityType creation operation.
print(movies_entity_type_lro.result())

name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/movies"
etag: "AMEw9yOxjDQ45socCP2SLRtRmxvAfa0oqDd78VUrmQKuLi4no0pM"



### Create Feature

You can also set a custom monitoring configuration at the Feature level, and view the properties and metrics in the console: sample [properties](https://storage.googleapis.com/cloud-samples-data/ai-platform-unified/datasets/featurestore/Feature%20Properties.png), sample [metrics](https://storage.googleapis.com/cloud-samples-data/ai-platform-unified/datasets/featurestore/Feature%20Snapshot%20Distribution.png).

In [15]:
# Create features for the 'users' entity.
# 'age' Feature leaves the monitoring config unset, which means it'll inherit the config from EntityType.
# 'gender' Feature explicitly disables monitoring.
# 'liked_genres' Feature is a STRING_ARRAY type, so it is automatically excluded from monitoring.
# For Features with monitoring enabled, distribution statistics are updated periodically in the console.
admin_client.batch_create_features(
    parent=admin_client.entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, "users"),
    requests=[
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.INT64,
                description="User age",
            ),
            feature_id="age",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING,
                description="User gender",
                monitoring_config=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig(
                    snapshot_analysis=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                        disabled=True,
                    ),
                ),
            ),
            feature_id="gender",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING_ARRAY,
                description="An array of genres that this user liked",
            ),
            feature_id="liked_genres",
        ),
    ],
).result()

features {
  name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/users/features/age"
  etag: "AMEw9yNhKHWFVrNDy8UGUb6bB-lUtt9uiJEvPYU654L3b6g8C1jB"
}
features {
  name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/users/features/gender"
  etag: "AMEw9yOxR6SMwJ8Viv2_OHWVBofxqe9IYhjItl5kAyb830bQ4rCS"
}
features {
  name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/users/features/liked_genres"
  etag: "AMEw9yN7epHeP-iZQzzDViV_F7r0rD8Vuta6DTu486dqcwA0o5qD"
}

In [16]:
# Create features for movies type.
# 'title' Feature enables monitoring.
admin_client.batch_create_features(
    parent=admin_client.entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, "movies"),
    requests=[
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING,
                description="The title of the movie",
                monitoring_config=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig(
                    snapshot_analysis=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                        monitoring_interval=Duration(seconds=172800),  # 2 days
                    ),
                ),
            ),
            feature_id="title",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING,
                description="The genres of the movie",
            ),
            feature_id="genres",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.DOUBLE,
                description="The average rating for the movie, range is [1.0-5.0]",
            ),
            feature_id="average_rating",
        ),
    ],
).result()

features {
  name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/movies/features/title"
  etag: "AMEw9yPsumOeGiCRw4hISl6ABAiD7qSfyEetva9bpmn5qhXM9Pw-"
}
features {
  name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/movies/features/genres"
  etag: "AMEw9yOwr-bHUVgv625Q_xDrXGIoQe9XAP0YakXx9gkjUfXCwJxv"
}
features {
  name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/movies/features/average_rating"
  etag: "AMEw9yMB2lIFR-wjjSXRvpyonp8k4t7LDLIzHSQWvRyt-ruKn-Bv"
}

## Search created features

While the [ListFeatures](https://cloud.google.com/vertex-ai/docs/featurestore/managing-featurestores#listing_featurestores) method allows you to easily view all features of a single
entity type, the [SearchFeatures](https://cloud.google.com/vertex-ai/docs/matching-engine) method searches across all featurestores
and entity types in a given location (such as `us-central1`). This can help you discover features that were created by someone else.

You can query based on feature properties including feature ID, entity type ID,
and feature description. You can also limit results by filtering on a specific
featurestore, feature value type, and/or labels.

In [17]:
# Search for all features across all featurestores.
list(admin_client.search_features(location=BASE_RESOURCE_PATH))

[name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/movies/features/average_rating"
 description: "The average rating for the movie, range is [1.0-5.0]"
 create_time {
   seconds: 1629803867
   nanos: 36566000
 }
 update_time {
   seconds: 1629803867
   nanos: 36566000
 },
 name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/movies/features/genres"
 description: "The genres of the movie"
 create_time {
   seconds: 1629803867
   nanos: 35305000
 }
 update_time {
   seconds: 1629803867
   nanos: 35305000
 },
 name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/movies/features/title"
 description: "The title of the movie"
 create_time {
   seconds: 1629803867
   nanos: 32685000
 }
 update_time {
   seconds: 1629803867
   nanos: 32685000
 },
 name: "projects/1066331464660/locations/us-central1/featurestores/movie_pred

Now, narrow down the search to features that are of type `DOUBLE`

In [18]:
# Search for all features with value type `DOUBLE`
list(
    admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=BASE_RESOURCE_PATH, query="value_type=DOUBLE"
        )
    )
)

[name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/movies/features/average_rating"
 description: "The average rating for the movie, range is [1.0-5.0]"
 create_time {
   seconds: 1629803867
   nanos: 36566000
 }
 update_time {
   seconds: 1629803867
   nanos: 36566000
 }]

Or, limit the search results to features with specific keywords in their ID and type.

In [19]:
# Filter on feature value type and keywords.
list(
    admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=BASE_RESOURCE_PATH, query="feature_id:title AND value_type=STRING"
        )
    )
)

[name: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/movies/features/title"
 description: "The title of the movie"
 create_time {
   seconds: 1629803867
   nanos: 32685000
 }
 update_time {
   seconds: 1629803867
   nanos: 32685000
 }]

## Import Feature Values

You need to import feature values before you can use them for online/offline serving. In this step, you will learn how to import feature values by calling the ImportFeatureValues API using the Python SDK.


### Source Data Format and Layout

As mentioned above, BigQuery table/Avro/CSV are supported. No matter what format you are using, each imported entity *must* have an ID; also, each entity can *optionally* have a timestamp, specifying when the feature values are generated. This notebook uses Avro as an input, located at this public [bucket](https://pantheon.corp.google.com/storage/browser/cloud-samples-data/ai-platform-unified/datasets/featurestore;tab=objects?project=storage-samples&prefix=&forceOnObjectsSortingFiltering=false). The Avro schemas are as follows:

**For the Users entity**:
```
schema = {
  "type": "record",
  "name": "User",
  "fields": [
      {
       "name":"user_id",
       "type":["null","string"]
      },
      {
       "name":"age",
       "type":["null","long"]
      },
      {
       "name":"gender",
       "type":["null","string"]
      },
      {
       "name":"liked_genres",
       "type":{"type":"array","items":"string"}
      },
      {
       "name":"update_time",
       "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
      },
  ]
 }
```

**For the Movies entity**
```
schema = {
 "type": "record",
 "name": "Movie",
 "fields": [
     {
      "name":"movie_id",
      "type":["null","string"]
     },
     {
      "name":"average_rating",
      "type":["null","double"]
     },
     {
      "name":"title",
      "type":["null","string"]
     },
     {
      "name":"genres",
      "type":["null","string"]
     },
     {
      "name":"update_time",
      "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
     },
 ]
}
```

### Import feature values for Users

When importing, specify the following in your request:

*   Data source format: BigQuery Table/Avro/CSV
*   Data source URL
*   Destination: featurestore/entity types/features to be imported


In [20]:
import_users_request = featurestore_service_pb2.ImportFeatureValuesRequest(
    entity_type=admin_client.entity_type_path(
        PROJECT_ID, REGION, FEATURESTORE_ID, "users"
    ),
# TODO 1a
# Make sure to replace your bucket name here.
    avro_source=io_pb2.AvroSource(
        # Source
        gcs_source=io_pb2.GcsSource(
            uris=[
                "gs://<Your-bucket-name>/users.avro"
            ]
        )
    ),
    entity_id_field="user_id",
    feature_specs=[
        # Features
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="age"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="gender"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="liked_genres"
        ),
    ],
    feature_time_field="update_time",
    worker_count=10,
)

In [21]:
# Start to import, will take a couple of minutes
ingestion_lro = admin_client.import_feature_values(import_users_request)

In [22]:
# Polls for the LRO status and prints when the LRO has completed
ingestion_lro.result()

imported_entity_count: 7
imported_feature_value_count: 12

### Import feature values for Movies

Similarly, import feature values for 'movies' into the featurestore.


In [23]:
import_movie_request = featurestore_service_pb2.ImportFeatureValuesRequest(
    entity_type=admin_client.entity_type_path(
        PROJECT_ID, REGION, FEATURESTORE_ID, "movies"
    ),
# TODO 1b
# Make sure to replace your bucket name here.
    avro_source=io_pb2.AvroSource(
        gcs_source=io_pb2.GcsSource(
            uris=[
                "gs://<Your-bucket-name>/movies.avro"
            ]
        )
    ),
    entity_id_field="movie_id",
    feature_specs=[
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="title"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="genres"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="average_rating"
        ),
    ],
    feature_time_field="update_time",
    worker_count=10,
)

In [24]:
# Start to import, will take a couple of minutes
ingestion_lro = admin_client.import_feature_values(import_movie_request)

In [25]:
# Polls for the LRO status and prints when the LRO has completed
ingestion_lro.result()

imported_entity_count: 4
imported_feature_value_count: 12

## Online serving

The
[Online Serving APIs](https://cloud.google.com/vertex-ai/docs/featurestore/serving-online)
lets you serve feature values for small batches of entities. It's designed for latency-sensitive service, such as online model prediction. For example, for a movie service, you might want to quickly shows movies that the current user would most likely watch by using online predictions.

### Read one entity per request

The ReadFeatureValues API is used to read feature values of one entity; hence
its custom HTTP verb is `readFeatureValues`. By default, the API will return the  latest value of each feature, meaning the feature values with the most recent  timestamp.

To read feature values, specify the entity ID and features to read. The response
contains a `header` and an `entity_view`. Each row of data in the `entity_view`
contains one feature value, in the same order of features as listed in the response header.

In [26]:
# Fetch the following 3 features.
feature_selector = FeatureSelector(
    id_matcher=IdMatcher(ids=["age", "gender", "liked_genres"])
)

# TODO 2a
data_client.read_feature_values(
    featurestore_online_service_pb2.ReadFeatureValuesRequest(
        # Fetch from the following feature store/entity type
        entity_type=admin_client.entity_type_path(
            PROJECT_ID, REGION, FEATURESTORE_ID, "users"
        ),
        # Fetch the user features whose ID is "alice"
        entity_id="alice",
        feature_selector=feature_selector,
    )
)

header {
  entity_type: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/users"
  feature_descriptors {
    id: "age"
  }
  feature_descriptors {
    id: "gender"
  }
  feature_descriptors {
    id: "liked_genres"
  }
}
entity_view {
  entity_id: "alice"
  data {
  }
  data {
  }
  data {
  }
}

### Read multiple entities per request

To read feature values from multiple entities, use the
StreamingReadFeatureValues API, which is almost identical to the previous
ReadFeatureValues API. Note that fetching only a small number of entities is recommended when using this API due to its latency-sensitive nature.

In [27]:
# Read the same set of features as above, but for multiple entities.
response_stream = data_client.streaming_read_feature_values(
# TODO 2b
    featurestore_online_service_pb2.StreamingReadFeatureValuesRequest(
        entity_type=admin_client.entity_type_path(
            PROJECT_ID, REGION, FEATURESTORE_ID, "users"
        ),
        entity_ids=["alice", "bob"],
        feature_selector=feature_selector,
    )
)

In [28]:
# Iterate and process response. Note the first one is always the header only.
for response in response_stream:
    print(response)

header {
  entity_type: "projects/1066331464660/locations/us-central1/featurestores/movie_prediction_20210824110930/entityTypes/users"
  feature_descriptors {
    id: "age"
  }
  feature_descriptors {
    id: "gender"
  }
  feature_descriptors {
    id: "liked_genres"
  }
}

entity_view {
  entity_id: "alice"
  data {
  }
  data {
  }
  data {
  }
}

entity_view {
  entity_id: "bob"
  data {
  }
  data {
  }
  data {
  }
}



Now that you have learned how to fetch imported feature values for online serving, the next step is learning how to use imported feature values for offline use cases.

## Batch Serving

Batch Serving is used to fetch a large batch of feature values for high-throughput, typically for training a model or batch prediction. In this section, you will learn how to prepare for training examples by calling the BatchReadFeatureValues API.

### Use case

**The task** is to prepare a training dataset to train a model, which predicts if a given user will watch a given movie. To achieve this, you need 2 sets of input:

*   Features: you already imported into the featurestore.
*   Labels: the groud-truth data recorded that user X has watched movie Y.


To be more specific, the ground-truth observation is described in Table 1 and the desired training dataset is described in Table 2. Each row in Table 2 is a result of joining the imported feature values from Feature Store according to the entity IDs and timestamps in Table 1. In this example,  the `age`, `gender` and `liked_genres` features from `users` and
the `genres` and `average_rating` features from `movies` are chosen to train the model. Note that only positive examples are shown in these 2 tables, i.e., you can imagine there is a label column whose values are all `True`.

BatchReadFeatureValues API takes Table 1 as
input, joins all required feature values from the featurestore, and returns Table 2 for training.

<h4 align="center">Table 1. Ground-truth Data</h4>

users | movies | timestamp            
----- | -------- | -------------------- 
alice  | Cinema Paradiso     | 2019-11-01T00:00:00Z 
bob  | The Shining     | 2019-11-15T18:09:43Z 
...   | ...      | ...     


<h4 align="center">Table 2. Expected Training Data Generated by Batch Read API (Positive Samples)</h4>

timestamp            | entity_type_users | age | gender | liked_genres | entity_type_movies | genres | average_rating  
-------------------- | ----------------- | --------------- | ---------------- | -------------------- | -------- | --------- | ----- 
2019-11-01T00:00:00Z | bob              | 35        | M                | [Action, Crime]                 | The Shining | Horror | 4.8 
2019-11-01T00:00:00Z | alice             | 55        | F                | [Drama, Comedy]                 | Cinema Paradiso | Romance | 4.5 
... | ... | ... | ... | ... | ... | ... | ... 
             

#### Why timestamp?

Note that there is a `timestamp` column in Table 2. This indicates the time when the ground-truth was observed. This is to avoid data inconsistency.

For example, the 1st row of Table 2 indicates that user `alice` watched movie `Cinema Paradiso` on `2019-11-01T00:00:00Z`. The featurestore keeps feature values for all timestamps but fetches feature values *only* at the given timestamp during batch serving. On 2019-11-01 alice might be 54 years old, but now alice might be 56; featurestore returns `age=54` as alice's age, instead of `age=56`, because that is the value of the feature at the observation time. Similarly, other features might be time-variant as well, such as liked_genres.

### Batch Read Feature Values

Assemble the request which specify the following info:

*   Where is the label data, i.e., Table 1.
*   Which features are read, i.e., the column names in Table 2.

The output is stored in a BigQuery table.


In [29]:
batch_serving_request = featurestore_service_pb2.BatchReadFeatureValuesRequest(
    # featurestore info
    featurestore=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
    # URL for the label data, i.e., Table 1.
    csv_read_instances=io_pb2.CsvSource(
        gcs_source=io_pb2.GcsSource(uris=[INPUT_CSV_FILE])
    ),
    destination=featurestore_service_pb2.FeatureValueDestination(
        bigquery_destination=io_pb2.BigQueryDestination(
            # Output to BigQuery table created earlier
            output_uri=DESTINATION_TABLE_URI
        )
    ),
    entity_type_specs=[
        featurestore_service_pb2.BatchReadFeatureValuesRequest.EntityTypeSpec(
            # Read the 'age', 'gender' and 'liked_genres' features from the 'users' entity
            # TODO 3a
            entity_type_id="users",
            feature_selector=FeatureSelector(
                id_matcher=IdMatcher(
                    ids=[
                        # features, use "*" if you want to select all features within this entity type
                        "age",
                        "gender",
                        "liked_genres",
                    ]
                )
            ),
        ),
        featurestore_service_pb2.BatchReadFeatureValuesRequest.EntityTypeSpec(
            # Read the 'average_rating' and 'genres' feature values of the 'movies' entity
            # TODO 3b
            entity_type_id="movies",
            feature_selector=FeatureSelector(
                id_matcher=IdMatcher(ids=["average_rating", "genres"])
            ),
        ),
    ],
)

In [30]:
# Execute the batch read
batch_serving_lro = admin_client.batch_read_feature_values(batch_serving_request)

In [31]:
# This long runing operation will poll until the batch read finishes.
batch_serving_lro.result()

After the LRO finishes, you should be able to see the result from the [BigQuery console](https://console.cloud.google.com/bigquery), in the dataset created earlier.

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

You can also keep the project but delete the featurestore:

In [32]:
admin_client.delete_featurestore(
    request=featurestore_service_pb2.DeleteFeaturestoreRequest(
        name=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        force=True,
    )
).result()
client.delete_dataset(
    DESTINATION_DATA_SET, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(DESTINATION_DATA_SET))

Deleted dataset 'movie_predictions_20210824110930'.
